# Templating input cells with [`jinja2`](http://jinja.pocoo.org/docs/2.10/)

This notebook demonstrates and exports the ability to place real into code cells before executing code. This idea can create parameterized reports.

    %reload_ext deathbeds.__Jinja2__

In [1]:
    from deathbeds.__Markdown__ import CallableTransformer, MarkdownImporter
    from jinja2 import Template
    from importnb import Notebook

* Use the callable transformer from the previous [Markdown input transformer](2018-07-06-Markdown-code-cells.ipynb)


In [2]:
    class TemplateTransformer(CallableTransformer):
        def __call__(self, str):
            from IPython import get_ipython
            return Template(str).render(**get_ipython().user_ns)

* Export the Ipython extension function.

In [3]:
    def load_ipython_extension(ip):
        ip.input_transformer_manager.physical_line_transforms.insert(0, TemplateTransformer())

## Importer

In [4]:
    import json

In [11]:
    class Jinja2Importer(Notebook):
        extensions = '.tpl.ipynb',
        def exec_module(self, module):
            with open(module.__file__) as file:
                for id, cell in enumerate(json.load(file)['cells']):
                    if cell['cell_type'] == 'code':
                        code = self.source_to_code(
                            self.format(Template(''.join(cell['source'])).render(**vars(module))),
                            module.__file__
                        )
                        exec(code, module.__dict__, module.__dict__)
                
            
            
        def parse_code_cell(self, cell, module=None): 
            cell['source'] = Template(''.join(cell['source'])).render(**vars(module))
            return super().parse_code_cell(cell)

### Proof of concept

* In the proof of concept it should be possible to combine the [Markdown transform](2018-07-06-Markdown-code-cells.ipynb) with the new template transformer.

In [5]:
    Ø = __name__ == '__main__'
    if Ø:
        %reload_ext deathbeds.__Markdown__
        load_ipython_extension(get_ipython())

* We can use this opportunity to test our Markdown importer too!

In [6]:
    if Ø:
        with MarkdownImporter():
            import deathbeds.__Jinja2__
            %reload_ext deathbeds.__Jinja2__
            
        assert deathbeds.__Jinja2__.__file__.endswith('.md.ipynb'), "Who the fuck knows where this file came from"

    if Ø:
        with MarkdownImporter():
            import deathbeds.__Jinja2__
            %reload_ext deathbeds.__Jinja2__
            
        assert deathbeds.__Jinja2__.__file__.endswith('.md.ipynb'), "Who the fuck knows where this file came from"


# Markdown and templates can now mix

    a = 10

> This is a template value of `a` which is {{a}}

    if Ø: from . import disqus